# Calibration of the Chaboche model with linear least squares

The goal of this example is to calibrate the simulator associated with the Chaboche model with linear least squares. 

In [1]:
import openturns as ot
import numpy as np
import CalibrationAnalysis as ca

## Read the observations

We begin by reading the observations from the data file. There are 100 observations of the couple (strain,stress).

In [2]:
observedSample = ot.Sample_ImportFromCSVFile("calibration_chaboche_observations.csv")
nbobs = observedSample.getSize()
strainobs = observedSample[:,0]
stressobs = observedSample[:,1]
nbobs

100

In [3]:
strainobs.setDescription(["Strain"])

## Define the model

We define the model which has 4 inputs and one output `stress`.

In [4]:
def modelChaboche(X):
    strain,R,C,gamma = X
    # Workaround for https://github.com/openturns/openturns/issues/1092
    R = R*1.e6
    C = C*1.e6
    stress = R + C*(1-np.exp(-gamma*strain))
    return [stress]

In [5]:
modelFunc = ot.PythonFunction(4, 1, modelChaboche)
modelFunc.setDescription(["strain", "R", "C", "gamma", "stress"])

Define the value of the reference values of the $\theta$ parameter. In the bayesian framework, this is called the mean of the *prior* gaussian distribution. In the data assimilation framework, this is called the *background*.

In [6]:
R = 700 # Exact : 750e6
C = 2500 # Exact : 2750e6
Gamma = 8. # Exact : 10
thetaPrior = ot.Point([R,C,Gamma])
thetaPrior.setDescription(("R","C","Gamma"))

The following statement create the calibrated function from the model. The calibrated parameters Ks, Zv, Zm are at indices 1, 2, 3 in the inputs arguments of the model.

In [7]:
calibratedIndices = [1,2,3]
mycf = ot.ParametricFunction(modelFunc, calibratedIndices, thetaPrior)

## Calibration

Defined the covariance matrix of the parameters $\theta$ to calibrate.

The `LinearLeastSquaresCalibration` class performs the linear least squares calibration by linearizing the model in the neighbourhood of the reference point.

In [8]:
algo = ot.LinearLeastSquaresCalibration(mycf, strainobs, stressobs, thetaPrior,"SVD")

The `run` method computes the solution of the problem.

In [ ]:
algo.run()

In [ ]:
calibrationResult = algo.getResult()

## Analysis of the results

The `getParameterMAP` method returns the maximum of the posterior distribution of $\theta$.

In [ ]:
thetaMAP = calibrationResult.getParameterMAP()
thetaMAP

class=Point name=Unnamed dimension=3 values=[747.008,2735.2,10.1292]

We can compute a 95% confidence interval of the parameter $\theta^\star$. 

In [ ]:
thetaPosterior = calibrationResult.getParameterPosterior()
thetaPosterior.computeBilateralConfidenceIntervalWithMarginalProbability(0.95)[0]

We can see that all three parameters are estimated with a large confidence interval.

In [ ]:
mypcr = ca.CalibrationAnalysis(calibrationResult, mycf, strainobs, stressobs)

In [ ]:
graph = mypcr.drawObservationsVsInputs()
graph.setLegendPosition("topleft")
graph

We see that there is a good fit after calibration, since the predictions after calibration (i.e. the green crosses) are close to the observations (i.e. the blue crosses).

In [ ]:
mypcr.drawObservationsVsPredictions()

We see that there is a much better fit after calibration, since the predictions are close to the diagonal of the graphics.

In [ ]:
observationError = calibrationResult.getObservationsError()
observationError

In [ ]:
graph = mypcr.drawResiduals()
graph.setLegendPosition("topleft")
graph

The analysis of the residuals shows that the distribution is centered on zero and symmetric. This indicates that the calibration performed well. 

Moreover, the distribution of the residuals is close to being gaussian. 

In [ ]:
_ = mypcr.drawThetaDistribution()